## 15. GPT4All Falcon 7B

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch

# Configuração do dispositivo (GPU se disponível, caso contrário CPU)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = "cuda:1"  # the device to load the model onto

model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/gpt4all-falcon'
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map="auto")#.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
####################################################################  COMPREENSÃO E GERAÇÃO DE TEXTO  ####################################################################
####################################################################  AVALIAÇÃO DA SEMÂNTICA PT-BR  ####################################################################

In [2]:
################# Question Answering com Contexto de Domínio #################

#############################  MELHOR RESULTADO  #############################

# Contexto e questão
context = r"""
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.
"""

question = """
Com base no texto, explique qual é o investimento mais adequado para alguém que busca proteção contra a inflação?
Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Passar ambos para o modelo e gerar a resposta
generate_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True, 
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investi

In [16]:
################# Question Answering de Simples Compreensão de Texto #################

context = r"""
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.
"""
question = "O que é a Grande Barreira de Corais e onde ela está localizada?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.


Q: O que é a Grande Barreira de Corais e onde ela está localizada?
A: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália.


In [3]:
################# Question Answering de Simples Compreensão de Texto ####.input_ids.to('cuda')#############

context = r"""
Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!
"""
question = "Resuma o parágrafo apresentado"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corp

In [18]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


KeyboardInterrupt: 

########################################  INFERÊNCIA LÓGICA  ########################################

In [19]:
################# Lógica Formal (do MMLU Benchmark) #################

context = r""" 
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.
"""

question = """
Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes

"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Passar ambos para o modelo e gerar a resposta
generate_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True, 
    temperature=0.8
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.


Q: 
Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes


A:


In [20]:
################# Dedução Lógica de Intenções #################

context = r"""
<<Exemplos>>
Exemplo 1:
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Categoria: Medicina

Exemplo 2:
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Categoria: Finanças
"""

question = """
Exemplo 3:
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- Categoria: ?
Substitua a incógnita '?' por um único termo definidor da categoria da frase do Exemplo 3
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Passar ambos para o modelo e gerar a resposta
generate_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True, 
    temperature=0.8
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



<<Exemplos>>
Exemplo 1:
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Categoria: Medicina

Exemplo 2:
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Categoria: Finanças


Q: 
Exemplo 3:
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- Categoria: ?
Substitua a incógnita '?' por um único termo definidor da categoria da frase do Exemplo 3

A: Educação on-line


In [21]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
    Observação: o prompt original teve que ser muito alterado para deixar explícita a intenção da questão, do contrário, nos testes empíricos o modelo (você) teve muita dificuldade em compreender a lógica. \
        Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação a inferência lógica?"

generate_ids
# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Passar ambos para o modelo e gerar a resposta
generate_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True, 
    temperature=0.8
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0])
print(answer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.     Observação: o prompt original teve que ser muito alterado para deixar explícita a intenção da questão, do contrário, nos testes empíricos o modelo (você) teve muita dificuldade em compreender a lógica.         Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação a inferência lógica?
A: Sim, para a questão das lógicas e sistemas de justiça, o modelo que eu construí tem dificuldades em inferir o que é justo ou verdadeiro em um mundo complexo e incerto. Os sistemas de justiça são complexos e muitas vezes não disponibilizam todas as informações necessárias para uma decisão justa ou verdadeira, o que dificulta a inferência de lógicas para esse tipo de contexto. O modelo também enfrenta problemas de incertezas, como os imprevistos ou as variáveis imprevisíveis.<|endoftext|>


In [22]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = r"""
<<Demonstração>>
Pergunta: Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
  Resposta Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.
"""

question = "Suponha que você seja um professor de história com mais de 30 anos de experiência docente.\
Instruções: Escreva uma pergunta de múltipla escolha para uma prova de história. A pergunta deve ser diferente da pergunta da <<Demonstração>>.\
Relevância: A questão deveria estar relacionada ao Renascimento.\
Restrição: Deve haver mais de duas respostas corretas entre as quatro opções. Oriente o aluno a assinalá-las."

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Passar ambos para o modelo e gerar a resposta
generate_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True, 
    temperature=0.8
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0])
print(answer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



<<Demonstração>>
Pergunta: Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
  Resposta Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.


Q: Suponha que você seja um professor de história com mais de 30 anos de experiência docente.Instruções: Escreva uma pergunta de múltipla escolha para uma prova de história. A pergunta deve ser diferente da pergunta da <<Demonstração>>.Relevância: A questão deveria estar relacionada ao Renascimento.Restrição: Deve haver mais de duas respostas corre

In [2]:
#################  ICL com self-reflection  #################

context = r""" """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=200,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0])
print(answer)

/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: Em relação à compreensão de texto, eu reconheço que tenho dificuldades em manter a atenção e concentração, assim como em compreender as nuances e subtilezas das palavras e frases. Também tenho dificuldades em relacionar o texto com o contexto e a realidade, e em compreender as ideias e argumentos apresentados.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: Em relação à compreensão de texto, eu reconheço que tenho dificuldades em manter a


Não conseguiu

In [3]:
################# Categorização e Clustering / Agrupamento #################

context = r"""
"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"
"""

question = """
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos por critério de similaridade semântica entre elas.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Passar ambos para o modelo e gerar a resposta
generate_ids = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=256,
    do_sample=True, 
    temperature=0.8
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"


Q: 
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos por critério de similaridade semântica entre elas.

A: 
1. Acesse:
2. Aumentar:
3. Consumir:
4. Subir:
5. Ampliar:
6. Muito:
7. Melhorar:
8. Muito:
9. Subir:
10. Muito:
11. Melhorar:
12. Melhorar:
13. Melhorar:
14. Melhorar:
15. Melhorar:
16. Melhorar:
17. Melhorar:
18. Mel

In [4]:
################# Categorização e Clustering / Agrupamento #################

context = r"""
"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"
"""

question = """
Esta é uma tarefa de agrupamento de frases similares.
Crie categorias semânticas e agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade entre elas.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)
################# Clustering ou Agrupamento #################

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"


Q: 
Esta é uma tarefa de agrupamento de frases similares.
Crie categorias semânticas e agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade entre elas.

A:
1. "As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
2. "Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
3. "O modelo

In [5]:
################# Categorização e Clustering / Agrupamento #################

context = r"""
"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"
"""

question = """
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde" 
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"


Q: 
Esta é uma tarefa de agrupamento de frases similares.
Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.

A:
1. "As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
2. "Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
3. "O modelo mais recente do iP

Não conseguiu completar satisfatoriamente a tarefa

In [6]:
#################  ICL com self-reflection  #################

context = r""" """
question = """
Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. 
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à tarefa requisitada?
"""



# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


 

Q: 
Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. 
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à tarefa requisitada?

A:

1. A autoavaliação de nota é de 0.00 a 1.00.
2. As dificuldades ou limitações que reconheço em relação à tarefa requisitada são:
* A necessidade de ter conhecimento de diversas áreas de informática, como sistemas operacionais, redes, eletrônica, eletrônica de potência, eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de potência eletrônica de po

In [7]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = r"""
Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!
"""

question = """
Resuma o parágrafo apresentado, em português evidentemente
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corp

In [8]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = r"""
"Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa
"""

question = """
Resuma o texto apresentado, discorrendo acerca de literatura e artes
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



"Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa


Q: 
Resuma o texto apresentado, discorrendo acerca de literatura e artes

A: Literatura é a maneira mais agradável de ignorar a vida. A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se a

In [9]:
####################################################################  TRANSFORMAÇÃO DE FORMATOS  ####################################################################

In [10]:
################# Machine Translation #################


context = r"""
Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 
"""

question = """
Traduza o parágrafo para o português brasileiro, o mais literalmente possível
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 


Q: 
Traduza o parágrafo para o português brasileiro, o mais literalmente possível

A: 
Tales from Shakespeare é um livro infantil escrito por Charles Lamb e sua irmã Mary Lamb em 1807. O livro é destinado a tornar as histórias de Shakespeare conhecidas para as crianças mais jovens. No entanto, como mencionado no Prefácio do autor, "as palavras de Shakespe

Dificuldade em entender que era para Traduzir, não interpretar/explicar

In [11]:
################# TRANSORMAÇÃO DE FORMATOS E AVALIAÇÃO DE SENSO COMUM #################


context = r"""
As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 

"""

question = """
Traduza o texto apresentado para o português brasileiro
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 



Q: 
Traduza o texto apresentado para o português brasileiro

A: 
O vendedor, um vendedor de enciclopedias, chegou ao local onde a casa do hermit, o hermit, ficava. Ele viu uma placa que dizia: "Nós não aceitamos vendedores. Terezanos entrarão em risco. Procurarão seus próprios riscos."

Embora o vendedor não tivesse sido convidado para entrar, ele ignorou a placa e dirigiu-se para a casa. Ao girar para a entrada, uma explosão poderosa na estrada o feriu.


In [12]:
################# TRANSORMAÇÃO DE FORMATOS E AVALIAÇÃO DE SENSO COMUM #################


context = r"""
Can Seller recover damages from Hermit for his injuries?
(A) Yes, unless Hermit, when he planted the charge, intended only to deter, not harm, intruders.
(B) Yes, if Hermit was responsible for the explosive charge under the driveway.
(C) No, because Seller ignored the sign, which warned him against proceeding further.
(D) No, if Hermit reasonably feared that intruders would come and harm him or his family.

"""

question = """
Selecione a alternativa correta entre as quatro apresentadas (A, B, C ou D) e justifique.

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') # Certificando de que os IDs estão no dispositivo correto (e no mesmo que o modelo)

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Can Seller recover damages from Hermit for his injuries?
(A) Yes, unless Hermit, when he planted the charge, intended only to deter, not harm, intruders.
(B) Yes, if Hermit was responsible for the explosive charge under the driveway.
(C) No, because Seller ignored the sign, which warned him against proceeding further.
(D) No, if Hermit reasonably feared that intruders would come and harm him or his family.



Q: 
Selecione a alternativa correta entre as quatro apresentadas (A, B, C ou D) e justifique.


A:
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A
A

In [13]:
#################################################################### RETIRAR REFERÊNCIAS DO TEXTO #################################################################### 

In [14]:
################# Retirar Referências do Texto #################

context = r"""
prompt = '''

Perceba a diferenca entre cada contexto sujo e contexto limpo contido em <<Exemplos>>
Aplique o mesmo raciocinio em <<Resposta>>
Contexto limpo: em <<Resposta>> deve estar limpo de referencias do tipo [algo]

<<Exemplos>>
Contexto sujo: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Contexto limpo: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.

Contexto sujo: Ontem foi dia de ramos [32].
Contexto limpo: Ontem foi dia de ramos.
"""

question = """

<<Resposta>>
Contexto sujo:
Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes.
A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27]
A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28]
Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15]
Considerava-a sua mãe de criação, e continuaria a chamá-la, por afeto, de "dadama" mesmo já adulto.[3][29]
A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.[28][30]
Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida.
José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33]
D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35]
Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36]
Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37]
Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38]
D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]
O imperador teve uma infância solitária e infeliz.[8][42]
A perda súbita de seus pais o assombraria por toda a vida;[43] ele teve poucos amigos de sua idade[28][36][44] e o contato com suas irmãs era limitado.[31][34][44]
O ambiente em que foi criado o tornou tímido e carente,[45][46] enxergando nos livros refúgio e fuga do mundo real.

Contexto limpo:
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do promptpara o português brasileiro
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=2000,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



prompt = '''

Perceba a diferenca entre cada contexto sujo e contexto limpo contido em <<Exemplos>>
Aplique o mesmo raciocinio em <<Resposta>>
Contexto limpo: em <<Resposta>> deve estar limpo de referencias do tipo [algo]

<<Exemplos>>
Contexto sujo: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Contexto limpo: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.

Contexto sujo: Ontem foi dia de ramos [32].
Contexto limpo: Ontem foi dia de ramos.


Q: 

<<Resposta>>
Contexto sujo:
Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes.
A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27]
A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28]
Quando bebê, D. Pedro II a chama

In [15]:
################# Retirar Referências do Texto #################

context = r"""

Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "['número']".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
"""



question = """
Contexto: Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes. A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27] A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28] Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15] Considerava-a sua mãe de criação, e continuaria a chamá-la, por afeto, de "dadama" mesmo já adulto.[3][29] A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.[28][30] Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida. 
José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]
O imperador teve uma infância solitária e infeliz.[8][42] A perda súbita de seus pais o assombraria por toda a vida;[43] ele teve poucos amigos de sua idade[28][36][44] e o contato com suas irmãs era limitado.[31][34][44] O ambiente em que foi criado o tornou tímido e carente,[45][46] enxergando nos livros refúgio e fuga do mundo real.

Retire as referências do texto dado e escreva a Resposta:

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.




Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "['número']".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.


Q: 
Contexto: Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes. A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27] A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28] Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15] Considerava-a sua mãe de criação, e continuaria a chamá-la, por af

In [16]:
################# Retirar Referências do Texto #################

context = r"""
Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "[número]".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
Contexto: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming
to communicate coherently [1]. These developments have brought about a revolutionary transformation [2] by enabling the creation of Large Language Models (LLMs) that can approximate humanlevel performance on certain evaluation benchmarks [3], [4].
Resposta: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming to communicate coherently. These developments have brought about a revolutionary transforbibliográficasmation by enabling the creation of Large Language Models (LLMs) that can approximate humanlevel performance on certain evaluation benchmarks.
"""



question = """
Retire as referências do texto dado abaixo:

Contexto: Ao deixar o país, o imperador D.Pedro I selecionou três pessoas para cuidarem de seu filho e das filhas remanescentes. A primeira foi José Bonifácio de Andrada, seu amigo e líder influente da independência brasileira, nomeado tutor.[26][27] A segunda foi Mariana Carlota de Verna Magalhães Coutinho (depois Condessa de Belmonte), que detinha o cargo de aia desde o nascimento de Pedro II.[28] Quando bebê, D. Pedro II a chamava de "dadama", pois não pronunciava corretamente a palavra "dama".[15] Considerava-a sua mãe de criação, e continuaria a chamá-la, por afeto, de "dadama" mesmo já adulto.[3][29] A terceira pessoa escolhida foi Rafael, um veterano negro da Guerra da Cisplatina.[28][30] Rafael era um empregado do paço em quem D. Pedro I tinha profunda confiança e a quem pediu que olhasse por seu filho — pedido que Rafael levaria a termo pelo resto de sua vida. José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41] O imperador teve uma infância solitária e infeliz.[8][42] A perda súbita dos seus pais o assombraria por toda a vida;[43] ele teve poucos amigos de sua idade[28][36][44] e o contato com suas irmãs era limitado.[31][34][44] O ambiente em que foi criado o tornou tímido e carente,[45][46] enxergando nos livros refúgio e fuga do mundo real.

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1500,
    top_k=50,
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Esta é uma tarefa de retirada de referências bibliográficas do texto. Estas referências estão escritas no formato "[número]".

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
Contexto: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming
to communicate coherently [1]. These developments have brought about a revolutionary transformation [2] by enabling the creation of Large Language Models (LLMs) that can approximate humanlevel performance on certain evaluation benchmarks [3], [4].
Resposta: Large Language Models (LLMs) have emerged as cutting-edge artificial intelligence systems designed to process and generate text, aiming to communicate coherently. These developme

In [17]:
####################################################################  EXTRAÇÃO DE CONTEÚDO  ####################################################################

In [18]:
################# Reconhecimento e Extração de Entidades #################


context = r"""
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.
"""

question = """
Analise o parágrafo, identifique Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto 
e, em seguida, produza os resultados em formato json.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.


Q: 
Analise o parágrafo, identifique Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto 
e, em seguida, produza os resultados em formato json.

A:
{
"pessoa": [
{
"nome": "Tim Cook",
"função": "CEO da Apple"
},
{
"nome": "Steve Jobs",
"função": "fundador da Apple"
},
{
"nome": "Steve Wozniak",
"função": "co-fundador da Apple"
}
],
"lugar": [
{
"nome": "Cupertino, Califórnia",
"função": "local onde o evento da Apple ocorreu"
},
{
"nome": "San Francisco, Califórnia",
"função": "local onde a Apple é uma das maiores empresas do mundo"
},
{
"nome": "San Jose, Califórnia",
"função": "local onde a Apple é uma das maiores empresas do mundo"
}
],
"organ

In [19]:
################# Reconhecimento e Extração de Entidades #################


context = r"""
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.
"""

question = """
Analise o parágrafo, identifique Somente as entities dos tipos "pessoa", "lugar" e "organização" mencionadas no texto e, em seguida, produza os resultados em formato json.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.


Q: 
Analise o parágrafo, identifique Somente as entities dos tipos "pessoa", "lugar" e "organização" mencionadas no texto e, em seguida, produza os resultados em formato json.

A:
{
"pessoa": [
{
"nome": "Tim Cook",
"função": "CEO da Apple"
},
{
"nome": "Steve Jobs",
"função": "fundador da Apple"
},
{
"nome": "Steve Wozniak",
"função": "co-fundador da Apple"
}
],
"lugar": [
{
"nome": "Cupertino, Califórnia",
"função": "local onde o evento da Apple ocorreu"
},
{
"nome": "Steve Jobs Theatre",
"função": "nome do teatro onde o evento da Apple ocorreu"
},
{
"nome": "Cupertino, Califórnia",
"função": "local onde a Apple está localizada"
}
],
"organização": [
{
"nome": "Apple",
"função": "nome da e

In [20]:
################# Reconhecimento e Extração de Entidades #################


context = r"""
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.
"""

question = """
Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.


Q: 
Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.

A:
{
"pessoa": [
{
"nome": "Tim Cook",
"função": "CEO da Apple"
},
{
"nome": "Steve Jobs",
"função": "fundador da Apple"
},
{
"nome": "Steve Wozniak",
"função": "co-fundador da Apple"
}
],
"lugar": [
{
"nome": "Cupertino, Califórnia",
"função": "local onde o evento ocorreu"
},
{
"nome": "Steve Jobs Theatre",
"função": "nome do teatro onde o evento ocorreu"
},
{
"nome": "Apple Park",
"função": "nome do novo campus de Apple"
}
],
"organização": [
{
"nome": "Apple",
"função": "nome da empresa"
},
{
"nome": "Steve Jobs",
"função": "fundador da 

In [21]:
################# Extração de Relações #################


context = r"""
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.
"""

question = """
Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to('cuda') 

# Geração da resposta
generate_ids = model.generate(
    input_ids=input_ids,
    max_new_tokens=1000,
    #temperature=0.8,
    top_k=50,
    #top_p=0.95
)

# Decodificação e impressão da resposta
answer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
print(answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.



Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.


Q: 
Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON

A:

The entities in the given sentence are "Elon Musk" and "SpaceX". The relationship between these entities is that "Elon Musk" is the founder of "SpaceX".


Resposta:

A:
{{
 "entities": [
 {
 "name": "Elon Musk",
 "relationships": [
 {
 "name": "SpaceX",
 "type": "founded"
 }
 ]
 },
 {
 "name": "SpaceX",
 "relationships": [
 {
 "name": "Elon Musk",
 "type": "founded"
 }
 ]
 }
 ],
 "relationships": [
 {
 "name": "Elon Musk",
 "type": "founded"
 },
 {
 "name": "SpaceX",
 "type": "founded"
 }
 ]
}


Estendeu muito a tarefa, investindo em estrutura ontológica. Verboso, mas correto.